In [214]:
import pandas as pd
import concurrent.futures
import requests
import time
import numpy as np
import glob2 as glob
from unidecode import unidecode

In [215]:
csv = pd.read_csv('annotations.csv')

In [216]:
csv['CM'] = (csv['configuracao_mao']).str.rsplit('/', 1).map(lambda x: x[1])
csv['CM'] = (csv['CM']).str.rsplit('.', 1).map(lambda x: x[0])

csv['filename'] = csv['filename'].map(lambda x:unidecode(x))
csv['filename'][652] = csv['filename'][652].rsplit('/', 1)[1]
csv['filename'][653] = csv['filename'][653].rsplit('/', 1)[1]

csv['palavra'] = csv['filename'].map(lambda x:unidecode(x))
csv.to_csv('processedAnnotations.csv', index=False)

In [223]:
CONNECTIONS = 30
TIMEOUT = 5
labels = np.unique(csv['CM'])
urls_CM = np.unique(csv['configuracao_mao'])
urls_videos = csv['video_url']
filenames = csv['filename']

In [218]:
def load_url_images(url, index, folder):
    r = requests.get(url, allow_redirects=True)
    open(folder + labels[index] + '.jpg', 'wb').write(r.content)
    
def load_url_videos(url, index, folder):
    r = requests.get(url, allow_redirects=True)
    open(folder + filenames[index], 'wb').write(r.content)
    
def getAllFiles (pathToFiles):
    files = []
    files = glob.glob(pathToFiles + '/*', recursive=True)
    return files

### Downloading video files

In [220]:
with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_url = {executor.submit(load_url_videos, url, index, "Videos/") : url for index, url in enumerate(urls_videos)}
    time1 = time.time()
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        finally:
            csv.loc[csv['video_url'] == url, 'downloaded'] = 'ok'
            print(str(len(out)),end="\r")
            
    time2 = time.time()

print(f'Took {time2-time1:.2f} s')
print(pd.Series(out).value_counts())

Took 69.37 s
Series([], dtype: int64)


In [224]:
csv[csv['downloaded'] != 'ok']

,palavra,video_url,filename,classe_gramatical,assunto,configuracao_mao,CM,downloaded


In [226]:
files = getAllFiles("/home/fabiana/Desktop/projeto-final-src/Crawler/Videos")
len(files)